# Predicting Optimal Fertilizer - XGBoost Implementierung

Nach der explorativen Datenanalyse mit Random Forest und Logistic Regression haben wir festgestellt, dass komplexere Modelle für bessere Ergebnisse notwendig sind. XGBoost (Extreme Gradient Boosting) ist bekannt für seine überlegene Performance bei strukturierten Daten und Kaggle-Competitions.

## Warum XGBoost?
- **Gradient Boosting**: Sequenzieller Aufbau schwacher Lerner (Decision Trees)
- **Regularisierung**: Integrierte L1/L2-Regularisierung verhindert Overfitting
- **Kategorische Features**: Native Unterstützung für kategorische Variablen
- **Performance**: Optimiert für Geschwindigkeit und Genauigkeit

## Imports und Konfiguration

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

Wir laden die benötigten Bibliotheken für XGBoost-Training, Evaluation und Visualisierung. Die Warnings werden unterdrückt, um eine saubere Ausgabe zu gewährleisten.

**Wichtige Bibliotheken:**
- `XGBClassifier`: Hauptmodell für Klassifizierung
- `StratifiedKFold`: Für ausbalancierte Cross-Validation
- `LabelEncoder`: Konvertierung der Zielvariable in numerische Form

## Datenvorbereitung

### Laden der Dateien
Wir laden drei Datensätze:
- **train.csv**: Kaggle-Trainingsdaten (700k Samples)
- **test.csv**: Kaggle-Testdaten für finale Vorhersagen
- **original.csv**: Original-Datensatz aus der Quelle

In [ ]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
original = pd.read_csv("data/Fertilizer Prediction.csv")

### Feature-Target Separation
- **Features (X)**: Alle Spalten außer 'id' und 'Fertilizer Name'
  - Temperatur, Humidity, Moisture (numerisch)
  - Soil Type, Crop Type (kategorisch)
  - Nitrogen, Potassium, Phosphorous (numerisch)
- **Target (y)**: 'Fertilizer Name' - 7 verschiedene Düngertypen

In [ ]:
# Drop the 'id' column
X = train.drop(columns=['id', 'Fertilizer Name'])

# Extract the target column
y = train['Fertilizer Name']

X.head()

### Label Encoding

In [ ]:
# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Output labels are now numbers
y_encoded[:10]

Die Zielvariable wird von String-Labels in numerische Werte umgewandelt:
- 14-35-14 → 0
- 10-26-26 → 1
- 17-17-17 → 2
- etc.

## Datenaugmentation Strategy
### Problem: Unbalancierte Klassen
Die Kaggle-Daten zeigen leichte Unbalancierung zwischen den Düngerklassen.
### Lösung: Original-Daten Integration
Wir vervielfachen den Original-Datensatz (n=6) und erreichen 700k zusätzliche Samples:
```python
for i in range(n):
    original = pd.concat([original, orig_copy], axis=0, ignore_index=True)

In [ ]:
orig_copy = original.copy()

n = 6
for i in range(n):
    original = pd.concat([original, orig_copy], axis=0, ignore_index=True)
    
original.info()

## Klassenverteilung Analyse

### Kaggle Trainingsdaten (750k Samples)
Analyse der Verteilung der 7 Düngerklassen im Kaggle-Datensatz:

In [ ]:
train['Fertilizer Name'].value_counts()

**Beobachtungen:**
- **Leichte Unbalancierung**: 14-35-14 (114k) vs Urea (92k) 
- **Verhältnis**: ~24% Unterschied zwischen häufigster und seltenster Klasse
- **Problem**: Kann zu Bias zugunsten häufigerer Klassen führen
- **Lösung**: Datenaugmentation mit Original-Daten notwendig

### Original-Datensatz (100k Samples)
Vergleich mit der Klassenverteilung im ursprünglichen Datensatz:

In [ ]:
orig_copy['Fertilizer Name'].value_counts()

**Vergleich Original vs Kaggle:**
- **Ausbalancierter**: Original-Daten zeigen gleichmäßigere Verteilung
- **Unterschiede**: 14-35-14 (14.5k) vs Urea (14.3k) - nur ~1% Differenz
- **Vorteil für Augmentation**: Original-Daten können Klassenunbalance ausgleichen
## Cross-Validation Setup und Initialisierung

Wir verwenden **Stratified K-Fold Cross-Validation** um eine robuste Modell-Evaluation zu gewährleisten:
- **5 Folds**: Standard für zuverlässige Performance-Schätzung
- **Stratified**: Erhält Klassenverteilung in jedem Fold
- **Shuffle=True**: Vermeidet systematische Bias durch Datenreihenfolge

Die Listen speichern Ergebnisse für spätere Ensemble-Bildung und finale Evaluation.

In [ ]:

map3_scores = []
models = []

all_y_true = []
all_y_pred = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## MAP@3 Metrik Implementation

**Mean Average Precision at 3** ist die Haupt-Evaluationsmetrik für diese Kaggle-Competition:

### Funktionsweise:
- Für jede Vorhersage werden die **Top-3 wahrscheinlichsten Klassen** betrachtet
- **Positionsgewichtung**: Höhere Position = höhere Gewichtung
- **Berechnung**: `1 / (Position der korrekten Klasse)` falls in Top-3, sonst 0


In [ ]:
def mapk(actual, predicted, k=3):
        def apk(a, p, k):
            if a in p[:k]:
                return 1.0 / (p[:k].index(a) + 1)
            return 0.0
        return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

## Cross-Validation Training Loop

Für jeden der 5 Folds führen wir folgende Schritte durch:
1. **Train/Validation Split** der Kaggle-Daten
2. **Datenaugmentation** durch Original-Daten
3. **Feature-Preprocessing** für XGBoost
4. **Model Training** mit Monitoring
5. **Evaluation** mit MAP@3 Metrik

In [ ]:
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y_encoded)):
    print(f"\nFold {fold + 1} ")

## Daten-Splitting und Augmentation pro Fold

### Train/Validation Split
- **Training**: 80% der Kaggle-Daten + komplette Original-Daten
- **Validation**: 20% der Kaggle-Daten (keine Augmentation!)

### Warum diese Strategie?
- **Mehr Trainingsdaten**: Original-Daten verbessern Modell-Robustheit
- **Saubere Evaluation**: Validation nur auf Kaggle-Daten testet echte Performance
- **Klassenbalance**: Original-Daten gleichen Unbalancierung aus

In [ ]:
    X_train = X.iloc[train_idx].copy()
    X_val = X.iloc[val_idx].copy()
    y_train = y_encoded[train_idx]
    y_val = y_encoded[val_idx]

    X_train = pd.concat([X_train, original], ignore_index=True)
    y_train = np.concatenate([y_train, le.transform(original['Fertilizer Name'])])

    X_train.drop(columns=['Fertilizer Name'], inplace=True)

## Feature Preprocessing für XGBoost

### Kategorische Feature-Behandlung
XGBoost kann **nativ mit kategorischen Features** umgehen, wenn sie korrekt als 'category' dtype markiert sind:

### Vorteile:
- **Keine One-Hot-Encoding** notwendig
- **Effizientere Speichernutzung**
- **Bessere Performance** bei hochkardinalischen Kategorien
- **Automatische Behandlung** unbekannter Kategorien

In [ ]:
    for col in X_train.columns:
        X_train[col] = X_train[col].astype('category')
        
    for col in X_val.columns:
        X_val[col] = X_val[col].astype('category')
    
    cat_features = X_train.columns.tolist()  

## XGBoost Model Definition und Training

### Hyperparameter-Erklärung:
- **max_depth=7**: Baumtiefe begrenzt Komplexität
- **learning_rate=0.01**: Kleine Schrittweite für stabile Konvergenz
- **n_estimators=10000**: Viele Bäume mit Early Stopping
- **Regularisierung**: alpha=2.7, lambda=1.4 verhindern Overfitting
- **Sampling**: subsample=0.8, colsample=0.4 für Robustheit

### Training-Monitoring:
- **eval_set**: Überwacht Training- und Validation-Loss
- **verbose=500**: Ausgabe alle 500 Iterationen

In [ ]:
    model = XGBClassifier(
                max_depth=7,
                colsample_bytree=0.4,
                subsample=0.8,
                n_estimators=10000,
                learning_rate=0.01,
                gamma=0.26,
                max_delta_step=4,
                reg_alpha=2.7,
                reg_lambda=1.4,
                objective='multi:softprob',
                random_state=13,
                enable_categorical=True,
                tree_method='hist',     
                device='cuda'  
            )

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train),(X_val, y_val)],
        verbose=500,
    )

## Vorhersagen und Performance-Evaluation

### Vorhersage-Typen:
- **y_pred**: Beste Klassen-Vorhersage (für F1-Score)
- **y_probs**: Wahrscheinlichkeiten für alle Klassen (für MAP@3)

### Top-3 Ranking:
- **np.argsort**: Sortiert Indizes nach Wahrscheinlichkeit
- **[:, -3:]**: Nimmt die 3 höchsten Werte
- **[:, ::-1]**: Kehrt Reihenfolge um (höchste zuerst)

In [ ]:
    y_pred = model.predict(X_val)
    y_probs = model.predict_proba(X_val)

    all_y_true.extend(y_val)
    all_y_pred.extend(y_pred)


    top3_preds = np.argsort(y_probs, axis=1)[:, -3:][:, ::-1]
    
    

    map3 = mapk(y_val.tolist(), top3_preds.tolist(), k=3)
    map3_scores.append(map3)
    models.append(model)

    print(f"F1 (macro): | MAP@3: {map3:.4f}")

## Cross-Validation Ergebnisse

### Performance-Zusammenfassung:
Die finale **durchschnittliche MAP@3** über alle 5 Folds gibt uns eine robuste Schätzung der Modell-Performance auf ungesehenen Daten.

### Interpretation:
- **MAP@3 ≈ 0.377**: Solide Performance für 7-Klassen-Problem
- **Konsistenz**: Geringe Varianz zwischen Folds zeigt Stabilität
- **Overfitting-Check**: Validation-Performance ist realistisch

In [ ]:
print("\n Final CV Results ")
print(f"Avg MAP@3: {np.mean(map3_scores):.4f}")

In [ ]:

results = model.evals_result()
plt.plot(results['validation_0']['mlogloss'], label='Train')
plt.plot(results['validation_1']['mlogloss'], label='Val')
plt.legend()
plt.show()

In [ ]:
for col in cat_features:
    test[col] = test[col].astype('category')

all_preds = np.zeros((test.shape[0], len(le.classes_)))

X_test = test.drop(columns='id')
cat_features = X_test.columns.tolist()   

for model in models:
    probs = model.predict_proba(X_test)
    all_preds += probs

avg_preds = all_preds / len(models)

top3_preds = np.argsort(probs, axis=1)[:, -3:][:, ::-1] 

top3_labels = le.inverse_transform(top3_preds.ravel()).reshape(top3_preds.shape)

submission = pd.DataFrame({
    'id': test['id'], 
    'Fertilizer Name': [' '.join(row) for row in top3_labels]
})

submission.to_csv('submission.csv', index=False)
submission.head()